In [8]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

## Read data from CSV

In [9]:
df = pd.read_csv(r'https://raw.githubusercontent.com/Jordwy/Data-Mining/refs/heads/main/Applied%20Practice%205%20-%20Data.csv')

## Seperate Data into Features and Class labels 

In [15]:
def pre_process(df: pd.DataFrame):

    # Seperate class from features
    features = df.drop('edible', axis=1)
    class_labels = df['edible']

    # Shuffle data
    features = features.sample(frac=1, random_state=1234)
    class_labels = class_labels.sample(frac=1, random_state=1234)

    # Seperate into test and train
    n = int(features.shape[0] * 0.8) # get 80% index num
    train_features, train_class_labels  = features.iloc[:n], class_labels.iloc[:n]
    test_features,  test_class_labels   = features.iloc[n:], class_labels.iloc[n:]

    return train_features, train_class_labels, test_features, test_class_labels




# Task 3 - Training a decision tree with Scikit-learn

**Exercise 1**: Create a function that takes dataframe as an input argument and converts all the
columns with “object” datatype, into categorical series with appropriate codes. 

*Hint: To convert a column of “object” dtype with string values to integer values you need to first convert into an accessor object for categorical properties of the pandas Series values and then you can use the series category attributes such as “codes”.*


In [11]:
def col_convert(df:pd.DataFrame) -> pd.DataFrame:
    '''Convert all columns of dtype 'object' into catagory codes (int)'''
    
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('category').cat.codes

    return df

In [ ]:
df = col_convert(df)
train_features, train_class_labels, test_features, test_class_labels = pre_process(df)
pre_process(df)

In [17]:
train_features

,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
362,2,2,4,0,5,1,1,0,4,1,...,2,7,7,0,2,1,0,2,0,1
1024,2,3,8,1,6,1,0,1,5,0,...,2,7,7,0,2,1,4,2,4,5
2272,2,0,3,1,5,1,0,0,5,1,...,2,7,6,0,2,1,4,2,4,0
3302,2,3,3,1,5,1,0,0,9,1,...,2,6,3,0,2,1,4,3,5,0
90,5,0,4,0,5,1,0,1,5,0,...,2,7,7,0,2,1,4,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,5,0,4,1,5,1,0,0,9,1,...,2,6,7,0,2,1,4,3,4,0
7748,5,0,8,0,5,1,1,0,10,0,...,1,7,7,0,2,2,4,7,2,1
8023,3,2,2,0,8,1,0,1,0,1,...,1,7,6,0,2,1,0,7,4,4
1563,5,0,4,1,5,1,0,0,7,1,...,2,6,3,0,2,1,4,3,4,0
